In [ ]:
import pandas as pd
import json

# --- 생활 습관 점수화 함수들 (100점 만점) ---
def calculate_smoking_score(tobacco):
    # 0은 비흡연을 의미하며, 만점인 38.0점으로 처리
    if tobacco == 0:
        return 38.0

    # None, NaN, 888 (이전 시스템의 "모름/무응답" 코드)는 0점으로 처리
    # 이 부분은 'tobacco'가 0이 아닌 경우에만 실행됩니다.
    if tobacco is None or pd.isna(tobacco) or tobacco == 888:
        return 0

    # 흡연 개비 수에 따른 ftnd_score_level 부여
    # 이 부분은 'tobacco'가 0도 아니고, None/NaN/888도 아닌,
    # 실제 개비 수(양의 정수)인 경우에만 실행됩니다.
    if tobacco <= 10: # 10개비 이하
        ftnd_score_level = 0
    elif tobacco <= 20: # 20개비 이하
        ftnd_score_level = 1
    elif tobacco <= 30: # 30개비 이하
        ftnd_score_level = 2
    else: # 30개비 초과
        ftnd_score_level = 3

    # 0점부터 38점까지 스케일링 (최저 0점, 최고 38점)
    # (3 - level) / 3 * max_score 공식을 적용합니다.
    return round(38.0 * (3 - ftnd_score_level) / 3, 1) # 소수점 첫째 자리에서 반올림



def calculate_walking_score(be3_33, be3_31):
    """걷기 시간 및 일수에 따른 점수 (28점 만점)"""
    if be3_31 is None or be3_33 is None or pd.isna(be3_31) or pd.isna(be3_33):
        return 0 # 둘 중 하나라도 없으면 0 점수

    # '안 함'이나 '비해당'이 매핑되어 0이 되는 경우 (map_user_raw_input에서 처리됨)
    # be3_31이 0이거나 be3_33이 0이면, 걷기 활동이 없는 것으로 간주
    if be3_31 == 0 or be3_33 == 0:
        return 0.0

    met = (be3_33) * 3.3 * be3_31 # METs 계산

    # IPAQ 활동 수준 → 원점수 (1: 낮음, 2: 보통, 3: 높음)
    if met >= 3000:
        base_score_level = 3 # 매우 활발
    elif met >= 600:
        base_score_level = 2 # 충분히 활발
    else:
        base_score_level = 1 # 어느 정도 활동

    # 0점부터 28점까지 스케일링 (level이 1일 때 0점, level이 3일 때 28점)
    # (level - 1) / 2 * max_score
    return round(28.0 * (base_score_level - 1) / 2, 1)

def calculate_alcohol_score(bd1_11, bd2_1):
    """음주 빈도 및 양에 따른 점수 (20점 만점)"""
    # Null/NaN 값 처리 (중간 점수 반환)
    if bd1_11 is None or bd2_1 is None or pd.isna(bd1_11) or pd.isna(bd2_1):
        return 0.0 # 20점 만점에 0

    # 음주 빈도 점수 (낮을수록 건강) - New logic
    if bd1_11 == 0: # 안 마심 (맵핑된 값 기준)
        freq_score = 0
    elif bd1_11 <= 12:  # 월1회 미만 또는 월1회정도 (맵핑된 값 기준)
        freq_score = 0 # Your base treats <=12 as the same as 0.0 for this score part
    elif bd1_11 <= 42:  # 월2~4회 (맵핑된 값 기준)
        freq_score = 1
    else:  # 주2회 이상 (맵핑된 값 기준)
        freq_score = 2

    # 1회 음주량 점수 (낮을수록 건강) - New logic
    # Assuming bd2_1 here refers to the *mapped* values (1.5, 3.5, 5.5, 8.0, 10.0)
    if bd2_1 <= 1.5: # Corresponds to 1-2잔 (original mapped 1.5)
        amount_score = 0
    elif bd2_1 <= 3.5: # Corresponds to 3-4잔 (original mapped 3.5)
        amount_score = 1
    else: # Corresponds to 5-6잔 or more (original mapped >=5.5)
        amount_score = 2

    # 총 위험 레벨 (0~4) - New logic
    total_risk_level = freq_score + amount_score

    # 총점 기반 건강 점수 (1~3점) - New logic
    if total_risk_level <= 1:
        base_score_level = 3 # Low risk, high score
    elif total_risk_level <= 3:
        base_score_level = 2
    else: # total_risk_level == 4
        base_score_level = 1 # High risk, low score

    # 20점까지 스케일링 (level이 1일 때 0점, level이 3일 때 20점)
    return round(20.0 * (base_score_level - 1) / 2, 1)

def calculate_breakfast_score(l_br_fq):
    """아침식사 빈도에 따른 점수 (15점 만점)"""
    if l_br_fq is None or pd.isna(l_br_fq):
        return 0 # 값이 없으면 0 점수

    # 빈도 기반 점수화 (높을수록 건강) - New logic (using mapped values)
    # l_br_fq values: 0.0 (안 함), 1.5 (1-2회), 3.5 (3-4회), 6.0 (5-7회)
    if l_br_fq >= 6.0: # 주 5~7회 (맵핑된 값 기준)
        base_score_level = 3  # Daily
    elif l_br_fq >= 3.5: # 주 3~4회 (맵핑된 값 기준)
        base_score_level = 2  # Frequent
    else: # 0.0 (안함) or 1.5 (1-2회)
        base_score_level = 1  # Infrequent or Not at all

    # 0점부터 15점까지 스케일링 (level이 1일 때 0점, level이 3일 때 15점)
    # (base_score_level - 1) / 2 * max_score
    return round(15.0 * (base_score_level - 1) / 2, 1)
#_______________________________________맵핑______________________________________________________________________
# --- 사용자 원시 데이터를 매핑하는 함수 ---
def map_user_raw_input(user_data_original):
    mapped_data = user_data_original.copy()
#_________________________________________________________________________________________________________________
    #tobacco 매핑
    tobacco_amount = mapped_data.get('tobacco')
    try:
        # 개비 수를 정수로 변환. 입력이 없거나 숫자가 아니면 None으로 처리
        mapped_data['tobacco'] = int(tobacco_amount) if tobacco_amount is not None and tobacco_amount != '' else None
    except (ValueError, TypeError): # Added TypeError for None
        mapped_data['tobacco'] = None # 숫자로 변환할 수 없는 경우
#____________________________________________________________________________________________________________________
    walk_freq_map = {
        1: 0,    # 사용자 값 1 -> 0(전혀 하지 않음)
        2: 1,    # 사용자 값 2 -> 1(1일)
        3: 2,   # 사용자 값 3 -> 2(2일)
        4: 3,   # 사용자 값 4 -> 3(3일)
        5: 4,  # 사용자 값 5 -> 4(4일)
        6: 5,  # 사용자 값 6 -> 5(5일)
        7: 6,  # 사용자 값 7 -> 6(6일)
        8: 7, # 사용자 값 8 -> 7(7일)
        99: None  # 모름/무응답
    }
    # get()을 사용하여 key가 없으면 원래 값을 사용
    mapped_data['BE3_31'] = walk_freq_map.get(user_data_original.get('BE3_31'), user_data_original.get('BE3_31'))


    # BE3_33 (1회 걷는 시간): 88 (안 함), 99 (모름/무응답)
    if mapped_data.get('BE3_33') in [88, 99]:
        mapped_data['BE3_33'] = None # None으로 처리하여 calculate_walking_score 함수 내에서 중간 점수 로직이 타도록 함
#_____________________________________________________________________________________________________________________
    # BD1_11 (1년간 음주 빈도): 설문 코드를 실제 빈도에 가깝게 매핑 (수정완)
    alcohol_freq_map = {
        1: 0,    # 사용자 값 1 -> 0 (최근 1년간 전혀 마시지 않았다)
        2: 6,    # 사용자 값 2 -> 0 (월1회 미만)
        3: 12,   # 사용자 값 3 -> 12 (월1회정도 (12개월에 12회))
        4: 42,   # 사용자 값 4 -> 42 (월2~4회 (월 평균 3.5회 * 12개월 = 42회))
        5: 130,  # 사용자 값 5 -> 130 (주2~3회 (주 평균 2.5회 * 52주 = 130회))
        6: 208,  # 사용자 값 6 -> 208 (주4회 (주 평균 4회 * 52주 = 208회))
        8: 0, # 사용자 값 8 -> 0 (마셔본적 없음)
        9: None  # 모름/무응답
    }
    # get()을 사용하여 key가 없으면 원래 값을 사용
    mapped_data['BD1_11'] = alcohol_freq_map.get(user_data_original.get('BD1_11'), user_data_original.get('BD1_11'))

    # BD2_1 (한 번에 마시는 음주량): 설문 코드를 잔 수 평균으로 매핑
    alcohol_amount_map = {
        1: 1.5,  # 사용자 값 1 -> 1.5 (1~2잔)
        2: 3.5,  # 사용자 값 2 -> 3.5 (3~4잔)
        3: 5.5,  # 사용자 값 3 -> 5.5 (5~6잔)
        4: 8.0,  # 사용자 값 4 -> 8.0 (7~9잔)
        5: 10.0, # 사용자 값 5 -> 10.0 (10잔 이상)
        8: 0, #사용자 값 9 -> 0 (비해당)
        9: None  # 모름/무응답
    }
    mapped_data['BD2_1'] = alcohol_amount_map.get(user_data_original.get('BD2_1'), user_data_original.get('BD2_1'))
#___________________________________________________________________________________________________________________
    # L_BR_FQ (아침식사 빈도): 설문 코드를 주당 횟수 평균으로 매핑
    breakfast_freq_map = {
        1: 6.0,  # 사용자 값 1 -> 6 (주 5~7회 (평균 6회))
        2: 3.5,  # 사용자 값 2 -> 3.5 (주 3~4회 (평균 3.5회))
        3: 1.5,  # 사용자 값 3 -> 1.5 (주 1~2회 (평균 1.5회))
        4: 0.0,  # 사용자 값 4 -> 0 (거의 안 먹음)
        9: None  # 모름/무응답
    }
    mapped_data['L_BR_FQ'] = breakfast_freq_map.get(user_data_original.get('L_BR_FQ'), user_data_original.get('L_BR_FQ'))

    return mapped_data
#______________________________________________________________________________________________________________
# --- 사용자 데이터 (기존과 동일) --- #사용자 입력 한
users = {
    "홍길동": {
        # "HE_glu": 100.0, "HE_HbA1c": 10.0, "HE_ht": 170, "HE_wt": 63, "HE_wc": 78,
        # "HE_BMI": 21.8, "HE_sbp1": 135, "HE_dbp1": 85, "HE_Upro": 3, "HE_chol": 200,
        # "HE_HDL_st2": 55, "HE_TG": 120, "HE_LDL_drct": 110, "HE_crea": 0.85,
        # "HE_ast": 22, "HE_alt": 18, "age": 20,


        "tobacco": 4,       # 일반 담배 하루 평균 흡연량 (숫자, 888/999 처리 필요)
        "BE3_33": 30,     # 1회 걷는 시간(분) (숫자, 88/99 처리 필요)
        "BE3_31": 8,      # 1주일간 걷기 일수 (설문 코드: 8은 매일)
        "BD1_11": 10,      # 1년간 음주빈도 (설문 코드: 3은 월1회정도)
        "BD2_1": 2,       # 한 번에 마시는 음주량 (설문 코드: 2는 3-4잔)
        "L_BR_FQ": 1      # 아침식사 빈도 (설문 코드: 1은 5~7회)
    }
}



# ----------------------------------------------------------------- 메인 실행 -----------------------------------------------------------------
if __name__ == "__main__":
    user_results = {}

    # 각 생활 습관 점수의 만점 상수 정의 (정확도를 위해 float 사용)
    MAX_SMOKING_SCORE = 38.0
    MAX_WALKING_SCORE = 28.0
    MAX_ALCOHOL_SCORE = 20.0
    MAX_BREAKFAST_SCORE = 15.0
    TOTAL_LIFESTYLE_MAX_SCORE_RAW = MAX_SMOKING_SCORE + MAX_WALKING_SCORE + MAX_ALCOHOL_SCORE + MAX_BREAKFAST_SCORE # 101.0

    # 사용자에게 보여줄 최종 종합 점수의 만점
    DISPLAY_TOTAL_SCORE_MAX = 100

    final_output_scores = {} # 최종 출력될 결과를 담을 딕셔너리

    for user_name, user_data_original in users.items():
        # 1. 원시 사용자 데이터를 매핑
        user_data_mapped = map_user_raw_input(user_data_original)

        # 2. 생활 습관 점수 계산
        smoking_score = calculate_smoking_score(user_data_mapped.get('BS3_2'))
        walking_score = calculate_walking_score(user_data_mapped.get('BE3_33'), user_data_mapped.get('BE3_31'))
        alcohol_score = calculate_alcohol_score(user_data_mapped.get('BD1_11'), user_data_mapped.get('BD2_1'))
        breakfast_score = calculate_breakfast_score(user_data_mapped.get('L_BR_FQ'))

        current_lifestyle_scores = []
        current_lifestyle_scores.append(smoking_score)
        current_lifestyle_scores.append(walking_score)
        current_lifestyle_scores.append(alcohol_score)
        current_lifestyle_scores.append(breakfast_score)

        # 3. 생활습관 총점 집계 및 100점 만점으로 환산
        avg_lifestyle_score_raw = sum(current_lifestyle_scores)

        if TOTAL_LIFESTYLE_MAX_SCORE_RAW > 0:
            # 100점 만점으로 스케일링
            final_lifestyle_score_scaled = round(avg_lifestyle_score_raw * (DISPLAY_TOTAL_SCORE_MAX / TOTAL_LIFESTYLE_MAX_SCORE_RAW), 1)
        else:
            final_lifestyle_score_scaled = 0.0

        # 최종 출력될 딕셔너리에 '생활습관_종합_점수'만 저장
        final_output_scores[user_name] = {
            f"생활습관_종합_점수 (/{int(DISPLAY_TOTAL_SCORE_MAX)}점 만점)": final_lifestyle_score_scaled
        }

    # 최종 결과를 JSON 형식으로 출력
    print(json.dumps(final_output_scores, indent=2, ensure_ascii=False))

{
  "홍길동": {
    "생활습관_종합_점수 (/100점 만점)": 48.5
  }
}


In [19]:
import pandas as pd
import json

# --- 생활 습관 점수화 함수들 (100점 만점) ---
def calculate_smoking_score(tobacco):
    # 0은 비흡연을 의미하며, 만점인 38.0점으로 처리
    if tobacco == 0:
        return 38.0

    # None, NaN, 888 (이전 시스템의 "모름/무응답" 코드)는 0점으로 처리
    # 이 부분은 'tobacco'가 0이 아닌 경우에만 실행됩니다.
    if tobacco is None or pd.isna(tobacco) or tobacco == 888:
        return 0

    # 흡연 개비 수에 따른 ftnd_score_level 부여
    # 이 부분은 'tobacco'가 0도 아니고, None/NaN/888도 아닌,
    # 실제 개비 수(양의 정수)인 경우에만 실행됩니다.
    if tobacco <= 10: # 10개비 이하
        ftnd_score_level = 0
    elif tobacco <= 20: # 20개비 이하
        ftnd_score_level = 1
    elif tobacco <= 30: # 30개비 이하
        ftnd_score_level = 2
    else: # 30개비 초과
        ftnd_score_level = 3

    # 0점부터 38점까지 스케일링 (최저 0점, 최고 38점)
    # (3 - level) / 3 * max_score 공식을 적용합니다.
    return round(38.0 * (3 - ftnd_score_level) / 3, 1) # 소수점 첫째 자리에서 반올림



def calculate_walking_score(be3_33, be3_31):
    """걷기 시간 및 일수에 따른 점수 (28점 만점)"""
    if be3_31 is None or be3_33 is None or pd.isna(be3_31) or pd.isna(be3_33):
        return 0 # 둘 중 하나라도 없으면 0 점수

    # '안 함'이나 '비해당'이 매핑되어 0이 되는 경우 (map_user_raw_input에서 처리됨)
    # be3_31이 0이거나 be3_33이 0이면, 걷기 활동이 없는 것으로 간주
    if be3_31 == 0 or be3_33 == 0:
        return 0.0

    met = (be3_33) * 3.3 * be3_31 # METs 계산

    # IPAQ 활동 수준 → 원점수 (1: 낮음, 2: 보통, 3: 높음)
    if met >= 3000:
        base_score_level = 3 # 매우 활발
    elif met >= 600:
        base_score_level = 2 # 충분히 활발
    else:
        base_score_level = 1 # 어느 정도 활동

    # 0점부터 28점까지 스케일링 (level이 1일 때 0점, level이 3일 때 28점)
    # (level - 1) / 2 * max_score
    return round(28.0 * (base_score_level - 1) / 2, 1)

def calculate_alcohol_score(bd1_11, bd2_1):
    """음주 빈도 및 양에 따른 점수 (20점 만점)"""
    # Null/NaN 값 처리 (중간 점수 반환)
    if bd1_11 is None or bd2_1 is None or pd.isna(bd1_11) or pd.isna(bd2_1):
        return 0.0 # 20점 만점에 0

    # 음주 빈도 점수 (낮을수록 건강) - New logic
    if bd1_11 == 0: # 안 마심 (맵핑된 값 기준)
        freq_score = 0
    elif bd1_11 <= 12:  # 월1회 미만 또는 월1회정도 (맵핑된 값 기준)
        freq_score = 0 # Your base treats <=12 as the same as 0.0 for this score part
    elif bd1_11 <= 42:  # 월2~4회 (맵핑된 값 기준)
        freq_score = 1
    else:  # 주2회 이상 (맵핑된 값 기준)
        freq_score = 2

    # 1회 음주량 점수 (낮을수록 건강) - New logic
    # Assuming bd2_1 here refers to the *mapped* values (1.5, 3.5, 5.5, 8.0, 10.0)
    if bd2_1 <= 1.5: # Corresponds to 1-2잔 (original mapped 1.5)
        amount_score = 0
    elif bd2_1 <= 3.5: # Corresponds to 3-4잔 (original mapped 3.5)
        amount_score = 1
    else: # Corresponds to 5-6잔 or more (original mapped >=5.5)
        amount_score = 2

    # 총 위험 레벨 (0~4) - New logic
    total_risk_level = freq_score + amount_score

    # 총점 기반 건강 점수 (1~3점) - New logic
    if total_risk_level <= 1:
        base_score_level = 3 # Low risk, high score
    elif total_risk_level <= 3:
        base_score_level = 2
    else: # total_risk_level == 4
        base_score_level = 1 # High risk, low score

    # 20점까지 스케일링 (level이 1일 때 0점, level이 3일 때 20점)
    return round(20.0 * (base_score_level - 1) / 2, 1)

def calculate_breakfast_score(l_br_fq):
    """아침식사 빈도에 따른 점수 (15점 만점)"""
    if l_br_fq is None or pd.isna(l_br_fq):
        return 0 # 값이 없으면 0 점수

    # 빈도 기반 점수화 (높을수록 건강) - New logic (using mapped values)
    # l_br_fq values: 0.0 (안 함), 1.5 (1-2회), 3.5 (3-4회), 6.0 (5-7회)
    if l_br_fq >= 6.0: # 주 5~7회 (맵핑된 값 기준)
        base_score_level = 3  # Daily
    elif l_br_fq >= 3.5: # 주 3~4회 (맵핑된 값 기준)
        base_score_level = 2  # Frequent
    else: # 0.0 (안함) or 1.5 (1-2회)
        base_score_level = 1  # Infrequent or Not at all

    # 0점부터 15점까지 스케일링 (level이 1일 때 0점, level이 3일 때 15점)
    # (base_score_level - 1) / 2 * max_score
    return round(15.0 * (base_score_level - 1) / 2, 1)
#_______________________________________맵핑______________________________________________________________________
# --- 사용자 원시 데이터를 매핑하는 함수 ---
def map_user_raw_input(user_data_original):
    mapped_data = user_data_original.copy()
#_________________________________________________________________________________________________________________
    # tobaccoType과 tobacco 매핑
    tobacco_amount = mapped_data.get('tobacco')
    try:
        # 개비 수를 정수로 변환. 입력이 없거나 숫자가 아니면 None으로 처리
        mapped_data['tobacco'] = int(tobacco_amount) if tobacco_amount is not None and tobacco_amount != '' else None
    except (ValueError, TypeError): # Added TypeError for None
        mapped_data['tobacco'] = None # 숫자로 변환할 수 없는 경우
#____________________________________________________________________________________________________________________
    walk_freq_map = {
        1: 0,    # 사용자 값 1 -> 0(전혀 하지 않음)
        2: 1,    # 사용자 값 2 -> 1(1일)
        3: 2,  # 사용자 값 3 -> 2(2일)
        4: 3,  # 사용자 값 4 -> 3(3일)
        5: 4,  # 사용자 값 5 -> 4(4일)
        6: 5,  # 사용자 값 6 -> 5(5일)
        7: 6,  # 사용자 값 7 -> 6(6일)
        8: 7, # 사용자 값 8 -> 7(7일)
        99: None  # 모름/무응답
    }
    # get()을 사용하여 key가 없으면 원래 값을 사용
    mapped_data['BE3_31'] = walk_freq_map.get(user_data_original.get('BE3_31'), user_data_original.get('BE3_31'))


    # BE3_33 (1회 걷는 시간): 88 (안 함), 99 (모름/무응답)
    if mapped_data.get('BE3_33') in [88, 99]:
        mapped_data['BE3_33'] = None # None으로 처리하여 calculate_walking_score 함수 내에서 중간 점수 로직이 타도록 함
#_____________________________________________________________________________________________________________________
    # BD1_11 (1년간 음주 빈도): 설문 코드를 실제 빈도에 가깝게 매핑 (수정완)
    alcohol_freq_map = {
        1: 0,    # 사용자 값 1 -> 0 (최근 1년간 전혀 마시지 않았다)
        2: 6,    # 사용자 값 2 -> 0 (월1회 미만)
        3: 12,   # 사용자 값 3 -> 12 (월1회정도 (12개월에 12회))
        4: 42,   # 사용자 값 4 -> 42 (월2~4회 (월 평균 3.5회 * 12개월 = 42회))
        5: 130,  # 사용자 값 5 -> 130 (주2~3회 (주 평균 2.5회 * 52주 = 130회))
        6: 208,  # 사용자 값 6 -> 208 (주4회 (주 평균 4회 * 52주 = 208회))
        8: 0, # 사용자 값 8 -> 0 (마셔본적 없음)
        9: None  # 모름/무응답
    }
    # get()을 사용하여 key가 없으면 원래 값을 사용
    mapped_data['BD1_11'] = alcohol_freq_map.get(user_data_original.get('BD1_11'), user_data_original.get('BD1_11'))

    # BD2_1 (한 번에 마시는 음주량): 설문 코드를 잔 수 평균으로 매핑
    alcohol_amount_map = {
        1: 1.5,  # 사용자 값 1 -> 1.5 (1~2잔)
        2: 3.5,  # 사용자 값 2 -> 3.5 (3~4잔)
        3: 5.5,  # 사용자 값 3 -> 5.5 (5~6잔)
        4: 8.0,  # 사용자 값 4 -> 8.0 (7~9잔)
        5: 10.0, # 사용자 값 5 -> 10.0 (10잔 이상)
        8: 0, #사용자 값 9 -> 0 (비해당)
        9: None  # 모름/무응답
    }
    mapped_data['BD2_1'] = alcohol_amount_map.get(user_data_original.get('BD2_1'), user_data_original.get('BD2_1'))
#___________________________________________________________________________________________________________________
    # L_BR_FQ (아침식사 빈도): 설문 코드를 주당 횟수 평균으로 매핑
    breakfast_freq_map = {
        1: 6.0,  # 사용자 값 1 -> 6 (주 5~7회 (평균 6회))
        2: 3.5,  # 사용자 값 2 -> 3.5 (주 3~4회 (평균 3.5회))
        3: 1.5,  # 사용자 값 3 -> 1.5 (주 1~2회 (평균 1.5회))
        4: 0.0,  # 사용자 값 4 -> 0 (거의 안 먹음)
        9: None  # 모름/무응답
    }
    mapped_data['L_BR_FQ'] = breakfast_freq_map.get(user_data_original.get('L_BR_FQ'), user_data_original.get('L_BR_FQ'))

    return mapped_data
#______________________________________________________________________________________________________________
# --- 사용자 데이터 (기존과 동일) --- #사용자 입력 한
users = {
    "홍길동": {
        # "HE_glu": 100.0, "HE_HbA1c": 10.0, "HE_ht": 170, "HE_wt": 63, "HE_wc": 78,
        # "HE_BMI": 21.8, "HE_sbp1": 135, "HE_dbp1": 85, "HE_Upro": 3, "HE_chol": 200,
        # "HE_HDL_st2": 55, "HE_TG": 120, "HE_LDL_drct": 110, "HE_crea": 0.85,
        # "HE_ast": 22, "HE_alt": 18, "age": 20,

        "tobacco": 20,         # 일반 담배 하루 평균 흡연량 (숫자, 888/999 처리 필요)
        "BE3_33": 60,       # 1회 걷는 시간(분) (숫자, 88/99 처리 필요)
        "BE3_31": 7,         # 1주일간 걷기 일수 (설문 코드: 8은 매일)
        "BD1_11": 1,         # 1년간 음주빈도 (설문 코드: 3은 월1회정도)
        "BD2_1": 4,          # 한 번에 마시는 음주량 (설문 코드: 2는 3-4잔)
        "L_BR_FQ": 5         # 아침식사 빈도 (설문 코드: 1은 5~7회)
    },

}


# ----------------------------------------------------------------- 메인 실행 -----------------------------------------------------------------
if __name__ == "__main__":
    user_results = {}

    # 각 생활 습관 점수의 만점 상수 정의 (정확도를 위해 float 사용)
    MAX_SMOKING_SCORE = 38.0
    MAX_WALKING_SCORE = 28.0
    MAX_ALCOHOL_SCORE = 20.0
    MAX_BREAKFAST_SCORE = 15.0
    TOTAL_LIFESTYLE_MAX_SCORE_RAW = MAX_SMOKING_SCORE + MAX_WALKING_SCORE + MAX_ALCOHOL_SCORE + MAX_BREAKFAST_SCORE # 101.0

    # 사용자에게 보여줄 최종 종합 점수의 만점
    DISPLAY_TOTAL_SCORE_MAX = 100

    final_output_scores = {} # 최종 출력될 결과를 담을 딕셔너리

    for user_name, user_data_original in users.items():
        # 1. 원시 사용자 데이터를 매핑
        user_data_mapped = map_user_raw_input(user_data_original)

        # 2. 생활 습관 점수 계산
        smoking_score = calculate_smoking_score(user_data_mapped.get('tobacco')) # Corrected line
        walking_score = calculate_walking_score(user_data_mapped.get('BE3_33'), user_data_mapped.get('BE3_31'))
        alcohol_score = calculate_alcohol_score(user_data_mapped.get('BD1_11'), user_data_mapped.get('BD2_1'))
        breakfast_score = calculate_breakfast_score(user_data_mapped.get('L_BR_FQ'))

        current_lifestyle_scores = []
        current_lifestyle_scores.append(smoking_score)
        current_lifestyle_scores.append(walking_score)
        current_lifestyle_scores.append(alcohol_score)
        current_lifestyle_scores.append(breakfast_score)

        # 3. 생활습관 총점 집계 및 100점 만점으로 환산
        avg_lifestyle_score_raw = sum(current_lifestyle_scores)

        if TOTAL_LIFESTYLE_MAX_SCORE_RAW > 0:
            # 100점 만점으로 스케일링
            final_lifestyle_score_scaled = round(avg_lifestyle_score_raw * (DISPLAY_TOTAL_SCORE_MAX / TOTAL_LIFESTYLE_MAX_SCORE_RAW), 1)
        else:
            final_lifestyle_score_scaled = 0.0

        # 최종 출력될 딕셔너리에 '생활습관_종합_점수'만 저장
        final_output_scores[user_name] = {
            f"생활습관_종합_점수 (/{int(DISPLAY_TOTAL_SCORE_MAX)}점 만점)": final_lifestyle_score_scaled
        }

    # 최종 결과를 JSON 형식으로 출력
    print(json.dumps(final_output_scores, indent=2, ensure_ascii=False))

{
  "홍길동": {
    "생활습관_종합_점수 (/100점 만점)": 56.2
  }
}
